In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as st

from pkg import detrend_group


In [3]:
yields = pd.read_csv("./data/yield_comparison.csv")
yields= yields.loc[:, ~yields.columns.str.contains("_lag|_lead|fao_idx|gridcells", regex=True)]

sm_tmax = pd.read_csv("./data/sm_tmax.csv")
yields_clim = yields.merge(sm_tmax, how="left", on = ["year", "cropname", "country"])

yields_clim = detrend_group(yields_clim, "sm_og", "sm_dt")
yields_clim = detrend_group(yields_clim, "tmax_og", "tmax_dt")

yields_clim = yields_clim[yields_clim.notna()]

In [10]:
test = yields_clim[["country", "cropname", "year", "sm_dt", "tmax_dt", "yield_log_dt", "csif_log_dt"]].sample(10000).dropna(how="any")
counts = test["country"].value_counts()
test = test[test["country"].isin(counts[counts > 10].index)].reset_index(drop=True)


In [28]:
def safe_regress(data, formula=None, yvar=None, xvars=None):
    if formula:
        yvar = formula.split("~")[0].strip()
        xvars = formula.split("~")[1].strip().split("+")
        xvars = [x.strip() for x in xvars]


    if formula is None and yvar is None or xvars is None:
        raise ValueError("Provide either formula or yvar + xvars")

    n_obs = len(data.dropna(subset=[yvar] + xvars))
    n_preds = len(xvars) + 1  # +1 for intercept

    if n_obs <= n_preds:
        return pd.DataFrame([{"r2": None, "adj_r2": None}])

    try:
        if formula:
            model = smf.ols(formula=formula, data=data).fit()
        else:
            formula = f"{yvar} ~ {' + '.join(xvars)}"
            model = smf.ols(formula=formula, data=data).fit()

        out = {"r2": model.rsquared,
            "adj_r2": model.rsquared_adj,
            "ftest_pval": model.f_pvalue}
        
        out.update({f"coef_{k}": v for k, v in model.params.items()})
        out.update({f"pval_{k}": v for k, v in model.pvalues.items()})
        
        ci = model.conf_int()
        for k in model.params.index:
            out[f"cilow_{k}"] = ci.loc[k, 0]
            out[f"cihigh_{k}"] = ci.loc[k, 1]

        return pd.DataFrame([out])

    except Exception as e:
        return pd.DataFrame([{"r2": None, "adj_r2": None, "error": str(e)}])


In [29]:
res = test.groupby(['cropname', 'country']).apply(
    lambda group: safe_regress(group, formula= "yield_log_dt ~ sm_dt + tmax_dt")
    ).reset_index(level=[0,1])
res = res.iloc[:, ~res.columns.str.contains("Intercept", regex=True)]


In [63]:
res.drop({"country"}, axis=1).groupby("cropname").quantile([0.25, 0.5, 0.75])



r2    adj_r2  ftest_pval  coef_sm_dt  \
cropname                                                                  
Barley                 0.25  0.212131 -0.266032    0.242715   -4.934035   
                       0.50  0.437507  0.093319    0.407192    2.438461   
                       0.75  0.744087  0.506280    0.679821   11.383815   
Cassava                0.25  0.287767 -0.217597    0.139919   -2.907171   
                       0.50  0.480069  0.154984    0.369595   -0.029285   
                       0.75  0.710839  0.573736    0.655056    5.476682   
Groundnuts, with shell 0.25  0.154655 -0.410608    0.200467   -2.220388   
                       0.50  0.356386 -0.094458    0.557565    2.115295   
                       0.75  0.718159  0.505259    0.783554    7.873017   
Maize                  0.25  0.265032 -0.199356    0.162178   -0.838046   
                       0.50  0.575439  0.249610    0.370914    3.527997   
                       0.75  0.842519  0.632631    0.636487   13.394668   
Millet                 0.25  0.330963 -0.103213    0.151880   -3.679451   
                       0.50  0.605801  0.289471    0.311697    0.982985   
                       0.75  0.819404  0.595683    0.575938    8.739604   
Oats                   0.25  0.278348 -0.254035    0.198969   -3.148717   
                       0.50  0.481188  0.120418    0.426349    2.006319   
                       0.75  0.727452  0.505683    0.658559    8.369685   
Potatoes               0.25  0.188076 -0.350656    0.261362   -2.131575   
                       0.50  0.410009 -0.079726    0.549523    1.535309   
                       0.75  0.696173  0.390300    0.737352    7.650873   
Pulses nes             0.25  0.216816 -0.326540    0.268881   -3.882625   
                       0.50  0.412478  0.045106    0.404336   -0.231085   
                       0.75  0.646948  0.378143    0.745280    4.839592   
Rapeseed               0.25  0.248426 -0.261477    0.189713   -4.194832   
                       0.50  0.462994  0.106962    0.477912    0.628585   
                       0.75  0.712788  0.450711    0.652042   12.412037   
Rice, paddy            0.25  0.189657 -0.235657    0.238773   -5.574077   
                       0.50  0.461071 -0.004106    0.517157   -0.057495   
                       0.75  0.730507  0.451647    0.685820    5.868660   
Rye                    0.25  0.169974 -0.253238    0.219286   -3.033243   
                       0.50  0.404427 -0.004091    0.483456    1.196209   
                       0.75  0.634516  0.378677    0.708365    8.611392   
Seed cotton            0.25  0.196992 -0.224438    0.231597   -4.804853   
                       0.50  0.399094 -0.058377    0.506043    0.380029   
                       0.75  0.634542  0.361324    0.703739    6.902248   
Sorghum                0.25  0.207592 -0.341760    0.209265   -3.567843   
                       0.50  0.451280  0.129146    0.381405    3.673430   
                       0.75  0.688639  0.394994    0.728480   12.783788   
Soybeans               0.25  0.264693 -0.205710    0.167615   -7.623196   
                       0.50  0.518493  0.163310    0.432470    0.465783   
                       0.75  0.815101  0.660749    0.630526    6.467622   
Sugar beet             0.25  0.176628 -0.216486    0.174213   -2.012723   
                       0.50  0.501441  0.147756    0.422422    2.409852   
                       0.75  0.762659  0.509142    0.657174   10.346999   
Sunflower seed         0.25  0.170073 -0.315249    0.191371   -5.813091   
                       0.50  0.435944 -0.007077    0.485054   -1.586413   
                       0.75  0.762447  0.591069    0.753816    3.542357   
Sweet potatoes         0.25  0.197967 -0.330626    0.272383   -3.805564   
                       0.50  0.401666 -0.027920    0.489414    0.178593   
                       0.75  0.638928  0.375429    0.728519    4.634496   
Wheat                  0.25  0.241835 -0.272791    0.145527 

In [11]:
res_csif = test.groupby(['cropname', 'country']).apply(
    lambda group: safe_regress(group, formula= "csif_log_dt ~ sm_dt + tmax_dt")
    ).reset_index(level=[0,1])
res_csif = res_csif.iloc[:, ~res_csif.columns.str.contains("Intercept", regex=True)]


In [13]:
res_csif.mean()

/var/folders/k4/2gcvspsj7d7c9hbxlvzv_zrh0000gn/T/ipykernel_13772/1660124230.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  res_csif.mean()


r2                 0.585361
adj_r2             0.254640
coef_sm_dt         4.104364
coef_tmax_dt       0.005299
pval_sm_dt         0.388011
pval_tmax_dt       0.482117
cilow_sm_dt      -28.448134
cihigh_sm_dt      36.656862
cilow_tmax_dt     -0.467475
cihigh_tmax_dt     0.478073
dtype: float64

In [22]:
testmod = smf.ols(data=test, formula="csif_log_dt~ sm_dt + tmax_dt").fit()
testmod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            csif_log_dt   R-squared:                       0.175
Model:                            OLS   Adj. R-squared:                  0.175
Method:                 Least Squares   F-statistic:                     963.9
Date:                Mon, 22 Sep 2025   Prob (F-statistic):               0.00
Time:                        11:47:28   Log-Likelihood:                 8895.9
No. Observations:                9061   AIC:                        -1.779e+04
Df Residuals:                    9058   BIC:                        -1.776e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0006      0.001      0.604      0.546      -0.001       0.002
sm_dt          4.2298      0.107     39.597      0.000       4.020       4.439
tmax_dt       -0.0006      0.002     -0.363      0.716      -0.004       0.002
==============================================================================
Omnibus:                     5439.311   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           516788.133
Skew:                          -2.003   Prob(JB):                         0.00
Kurtosis:                      39.780   Cond. No.                         112.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
print(testmod.f_pvalue)


0.0


In [ ]:

rm(list=ls())

df <- read_csv("./data/dt_clim_vars.csv")
df$iso_a3 <- as.factor(df$iso_a3)
df$cropname <- as.factor(df$cropname)

countries <- ne_countries(scale = "medium", returnclass = "sf") %>%
  st_transform(crs = "+proj=moll +lon_0=0") %>%
  filter(continent != "Antarctica") %>%
  mutate(iso_a3 = case_when(
    name == "France" ~ "FRA",
    name == "Norway" ~ "NOR",
    TRUE ~ iso_a3))

fit_model <- function(df, response_var = "csif") {
  formula_lm <- as.formula(paste(response_var, "~ sm_dt + tmax_dt"))
  
  if (nrow(df) <= 5) return(NULL)
  
  model <- try(lm(formula_lm, data = df), silent = TRUE)
  if (inherits(model, "try-error")) return(NULL)
  
  model_summary <- summary(model)
  r2 <- model_summary$r.squared
  adj_r2 <- model_summary$adj.r.squared
  coefs <- coef(model)
  coef_table <- model_summary$coefficients
  
  extract_coef <- function(name) {
    if (name %in% names(coefs)) coefs[[name]] else NA_real_}
  
  extract_pval <- function(name) {
    if (name %in% rownames(coef_table)) coef_table[name, "Pr(>|t|)"] else NA_real_}
  
  return(data.frame(
    cropname = df$cropname[1],
    iso_a3 = df$iso_a3[1],
    response_var = response_var,
    r2 = r2,
    adj_r2 = adj_r2,
    intercept = extract_coef("(Intercept)"),
    sm = extract_coef("sm_dt"),
    tmax = extract_coef("tmax_dt"),
    pval_sm = extract_pval("sm_dt"),
    pval_tmax = extract_pval("tmax_dt")
  ))}

result_csif <- df %>%
  group_by(cropname, iso_a3) %>%
  group_split() %>%
  lapply(fit_model, response_var = "csif") %>%
  bind_rows()

result_yield <- df %>%
  group_by(cropname, iso_a3) %>%
  group_split() %>%
  lapply(fit_model, response_var = "yield") %>%
  bind_rows()

result <- full_join(result_csif, result_yield, by=c("cropname", "iso_a3"))
result$response_var.x<- NULL
result$response_var.y<- NULL
colnames(result) <- gsub("\\.x$", "_csif", colnames(result)) 
colnames(result) <- gsub("\\.y$", "_census", colnames(result))  

save(result, file="./models/clim_linear.Rda")


map_linear <- countries %>%
  rename(country = name) %>%
  left_join(result, by = "iso_a3")

save(map_linear, file="./models/clim_linear_map.Rda")
  